In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [ ]:
# rm(list=ls()) 

# library(devtools)


# library(choroplethrZip)
# library(tidyverse)

# https://github.com/arilamstein/choroplethrZip

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,23dcb0d8-3e61-45bf-899f-b951946ce2ff
1,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,55783cdd-ccbd-4b19-930b-072def248507
2,0,2020-01-22T16:55:53.194Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,bfb5922b-1b21-4882-b1d4-b3825ff53e37
3,0,2019-09-19T13:45:58.425Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,c1cf6e00-1e6d-4493-93fc-eb430ef15495
4,0,2019-09-19T13:45:58.425Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,cad8334c-13f2-4837-bdcf-e09591197ff7


In [4]:
df['reportedZipCode'].dtypes

dtype('float64')

In [5]:
df['reportedZipCode'].isna().sum() / df.shape[0]

0.021602852983582808

In [6]:
df['censusBlockGroupFips'].isna().sum() / df.shape[0]

0.053005097819786226

In [7]:
df['latitude'].isna().sum() / df.shape[0]

0.015378203744076599

In [8]:
df['longitude'].isna().sum() / df.shape[0]

0.015378203744076599

In [9]:
missing_all_three = df['reportedZipCode'].isna() | df['censusBlockGroupFips'].isna() | df['latitude'].isna()

# To get the number of such rows
num_missing_all_three = missing_all_three.sum()

# To get the proportion of such rows
proportion_missing_all_three = num_missing_all_three / df.shape[0]

In [10]:
proportion_missing_all_three*100

6.149075821846406

In [11]:
#Zipcode list

list_zipcode = df['reportedZipCode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode = [zipcode.zfill(5) for zipcode in list_zipcode]

# temp = pd.DataFrame(list_censusBG, columns=['ZipCode'])

# temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/ZipCode.csv', index=False)

In [12]:
#CensusBG list

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

# temp = pd.DataFrame(list_censusBG, columns=['censusBG'])

# temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/censusBG.csv', index=False)

In [13]:
#lat long df

df_lat_long = df[['latitude', 'longitude']].dropna().drop_duplicates()

## Using pygris

In [ ]:
years = [2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [ ]:
# zip_codes_list = pygris.zctas(year=2000)

In [ ]:
# zip_codes_list.head()

In [ ]:
# zip_codes_list.shape[0]

In [ ]:
# years = [2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

years = [2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

yearly_data = {}

for i in years:
    yearly_data[i] = pygris.zctas(year=i)

In [ ]:
# temp = pygris.zctas(year=2000)

In [ ]:
temp = pd.DataFrame(yearly_data[2021])

In [ ]:
temp.head()

In [ ]:
temp = temp[['ZCTA5CE20', 'geometry']]

In [ ]:
temp = temp.rename(columns={"ZCTA5CE20": "ZIPcode"})
temp['year'] = 2021
temp

In [ ]:
list_zipcode_temp = temp['ZIPcode'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_temp = [zipcode.zfill(5) for zipcode in list_zipcode_temp]

In [ ]:
len(list_zipcode_temp)

In [ ]:
list_zipcode_3 = list_zipcode_3 + list_zipcode_temp

In [ ]:
len(list_zipcode_3)

In [ ]:
list_zipcode_3 = list(set(list_zipcode_3))

In [ ]:
len(list_zipcode_3)

In [ ]:
set_zipcode = set(list_zipcode)
set_zipcode_3 = set(list_zipcode_3)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_3)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(common_zipcodes)/len(list_zipcode)

In [ ]:
zip_code_df2000 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2000.head()

In [ ]:
zip_code_df2010 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2010.head()

In [ ]:
zip_code_df2012 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2012.head()

In [ ]:
zip_code_df2020 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2020.head()

In [ ]:
zip_code_df2021 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2021.head()

In [ ]:
zip_code_df = pd.concat([zip_code_df2000, zip_code_df2010, zip_code_df2012, zip_code_df2020, zip_code_df2021], ignore_index=True)

In [ ]:
zip_code_df = zip_code_df.drop_duplicates(subset=['ZIPcode', 'geometry'])

In [ ]:
zip_code_df.shape[0]

In [ ]:
zip_code_df.head()

In [ ]:
# Convert the geometry column to WKT format
zip_code_df['geometry'] = zip_code_df['geometry'].apply(lambda geom: geom.wkt)

In [ ]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(zip_code_df), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = zip_code_df.iloc[start:end].copy()
    # temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip", compression='gzip')

In [2]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_0_25000.parquet.gzip")

# Convert the WKT strings back to geometries
gdf_read = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [3]:
df_read.head()

,ZIPcode,geometry,year
0,38739,"MULTIPOLYGON (((-90.499253 34.053217, -90.4997...",2000
1,38767,"POLYGON ((-90.76477 34.119506, -90.766978 34.1...",2000
2,38669,"POLYGON ((-90.74080599999999 34.184565, -90.74...",2000
3,38630,"MULTIPOLYGON (((-90.67204699999999 34.259564, ...",2000
4,38639,"POLYGON ((-90.456458 34.31975, -90.456758 34.3...",2000


In [5]:
import geopandas as gpd
from shapely.wkt import loads

# Assuming you have a DataFrame df with columns 'unit' and 'geometry'
# Example:
#    unit  |  geometry
#   -------|-----------
#    unit1 |  geom1
#    unit2 |  geom2
#    unit3 |  geom3

# Here's a hypothetical dataset for demonstration
data = {
    'unit': ['unit1', 'unit2', 'unit3'],
    'geometry': ['POLYGON ((0 0, 0 2, 2 2, 2 0, 0 0))',
                'POLYGON ((1 1, 1 3, 3 3, 3 1, 1 1))',
                'POLYGON ((0.5 0.5, 0.5 2.5, 2.5 2.5, 2.5 0.5, 0.5 0.5))']
}

# Convert string representations of geometries to actual Shapely geometry objects
data['geometry'] = [loads(geom) for geom in data['geometry']]

df = gpd.GeoDataFrame(data, geometry='geometry')

# Get the intersection of the three geometries
intersection_geom = df['geometry'].iloc[0]
for geom in df['geometry'].iloc[1:]:
    intersection_geom = intersection_geom.intersection(geom)

print(intersection_geom)

POLYGON ((2 1, 1 1, 1 2, 2 2, 2 1))


In [ ]:
# Drop duplicates
zip_code_df = zip_code_df.drop_duplicates(subset=['ZIPcode'])

In [ ]:
list_zipcode_2 = zip_codes_list['ZCTA5CE20'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_2 = [zipcode.zfill(5) for zipcode in list_zipcode_2]

In [ ]:
# Convert both lists to sets
set_zipcode = set(list_zipcode)
set_zipcode_2 = set(list_zipcode_2)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_2)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(common_zipcodes)/len(list_zipcode)

In [14]:
states = pygris.states()
states_list = sorted(states['STUSPS'].unique())

Using the default year of 2021


In [ ]:
zip_codes_list['ZCTA5CE20'] = zip_codes_list['ZCTA5CE20'].astype(str)

filtered_df = zip_codes_list[zip_codes_list['ZCTA5CE20'].isin(list_zipcode)]

In [ ]:
filtered_df.head()

In [ ]:
years2 = [1990, 2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [15]:
BG_list = []

for i in states_list:
    BG_list.append(pygris.block_groups(state = i, year = 2021))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [16]:
BG_list_2012 = []

for i in states_list:
    BG_list_2012.append(pygris.block_groups(state = i, year = 2012))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [17]:
BG_list_2010 = []

for i in states_list:
    BG_list_2010.append(pygris.block_groups(state = i, year = 2010))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [18]:
BG_list_2000 = []

for i in states_list:
    BG_list_2000.append(pygris.block_groups(state = i, year = 2000))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [20]:
BG_list_1990 = []

for i in states_list:
    BG_list_1990.append(pygris.block_groups(state = i, year = 1990, cb = True))

Using FIPS code '02' for input 'AK'
Using FIPS code '01' for input 'AL'
Using FIPS code '05' for input 'AR'
Using FIPS code '60' for input 'AS'
Using FIPS code '04' for input 'AZ'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '11' for input 'DC'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '66' for input 'GU'
Using FIPS code '15' for input 'HI'
Using FIPS code '19' for input 'IA'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'
Using FIPS code '25' for input 'MA'
Using FIPS code '24' for input 'MD'
Using FIPS code '23' for input 'ME'
Using FIPS code '26' for input 'MI'
Using FIPS code '27' for input 'MN'
Using FIPS code '29' for input 'MO'
Using FIPS code '69' for inp

In [147]:
temp = pd.concat(BG_list_1990, ignore_index=True)

In [148]:
temp.head()

,AREA,PERIMETER,BG02_D90_,BG02_D90_I,POLY_,SUBCLASS,SUBCLASS_,RINGS_OK,RINGS_NOK,ST,CO,TRACT,BG,GEOID,AREALAND,AREAWAT,AREATOT,NAME,geometry,BG01_D90_,BG01_D90_I,BG05_D90_,BG05_D90_I,BG60_D90_,BG60_D90_I,BG04_D90_,BG04_D90_I,BG06_D90_,BG06_D90_I,BG08_D90_,BG08_D90_I,BG09_D90_,BG09_D90_I,BG11_D90_,BG11_D90_I,BG10_D90_,BG10_D90_I,BG12_D90_,BG12_D90_I,BG13_D90_,BG13_D90_I,BG66_D90_,BG66_D90_I,BG15_D90_,BG15_D90_I,BG19_D90_,BG19_D90_I,BG16_D90_,BG16_D90_I,BG17_D90_,BG17_D90_I,BG18_D90_,BG18_D90_I,BG20_D90_,BG20_D90_I,BG21_D90_,BG21_D90_I,BG22_D90_,BG22_D90_I,BG25_D90_,BG25_D90_I,BG24_D90_,BG24_D90_I,BG23_D90_,BG23_D90_I,BG26_D90_,BG26_D90_I,BG27_D90_,BG27_D90_I,BG29_D90_,BG29_D90_I,BG69_D90_,BG69_D90_I,BG28_D90_,BG28_D90_I,BG30_D90_,BG30_D90_I,BG37_D90_,BG37_D90_I,BG38_D90_,BG38_D90_I,BG31_D90_,BG31_D90_I,BG33_D90_,BG33_D90_I,BG34_D90_,BG34_D90_I,BG35_D90_,BG35_D90_I,BG32_D90_,BG32_D90_I,BG36_D90_,BG36_D90_I,BG39_D90_,BG39_D90_I,BG40_D90_,BG40_D90_I,BG41_D90_,BG41_D90_I,BG42_D90_,BG42_D90_I,BG72_D90_,BG72_D90_I,BG44_D90_,BG44_D90_I,BG45_D90_,BG45_D90_I,BG46_D90_,BG46_D90_I,BG47_D90_,BG47_D90_I,BG48_D90_,BG48_D90_I,BG49_D90_,BG49_D90_I,BG51_D90_,BG51_D90_I,BG78_D90_,BG78_D90_I,BG50_D90_,BG50_D90_I,BG53_D90_,BG53_D90_I,BG55_D90_,BG55_D90_I,BG54_D90_,BG54_D90_I,BG56_D90_,BG56_D90_I
0,0.030489,0.947504,2.0,1.0,2,BG_02SUB,1,1,0,02,185,950100,1,021859501001,110.099,277.052,387.151,BG 1,"POLYGON ((-156.77553 71.29742, -156.77612 71.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.036164,0.936381,3.0,2.0,3,BG_02SUB,2,1,0,02,185,950100,3,021859501003,140.170,96.286,236.456,BG 3,"POLYGON ((-156.80670 71.28787, -156.80565 71.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000117,0.072402,4.0,3.0,4,BG_02SUB,3,1,0,02,185,950100,2,021859501002,0.458,7.978,8.436,BG 2,"POLYGON ((-156.77553 71.29742, -156.78188 71.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11.554386,28.048671,5.0,4.0,5,BG_02SUB,4,1,0,02,185,950200,2,021859502002,48124.400,4663.000,52787.400,BG 2,"POLYGON ((-156.38962 71.27306, -156.35948 71.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.685307,18.542956,6.0,5.0,6,BG_02SUB,5,1,0,02,185,950200,1,021859502001,22923.700,2743.200,25666.900,BG 1,"POLYGON ((-154.31142 70.82173, -154.27104 70.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [149]:
temp = temp[['GEOID', 'geometry']]
temp['year'] = 1990

temp = temp.rename(columns={"GEOID": "GEOID"})

In [150]:
list_censusBG_2 = temp['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG_2 = [censusBG.zfill(12) for censusBG in list_censusBG_2]

In [151]:
len(list_censusBG_2)

228588

In [152]:
list_censusBG_3 = list_censusBG_3 + list_censusBG_2 

In [153]:
len(list_censusBG_3)

577465

In [154]:
list_censusBG_3 = list(set(list_censusBG_3))

In [155]:
len(list_censusBG_3)

410788

In [156]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [157]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_3)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 106781


In [158]:
len(common_bg)/len(list_censusBG)

0.999953177382803

In [159]:
len(common_bg)

106781

In [65]:
BG_list_df2021 = temp[temp['GEOID'].isin(common_bg)]

In [66]:
BG_list_df2021.head()

,GEOID,geometry,year
9,020900009002,"POLYGON ((-147.96856 64.77841, -147.96852 64.7...",2021
12,021300002002,"POLYGON ((-131.70187 55.38841, -131.70029 55.3...",2021
16,020200021003,"POLYGON ((-149.92843 61.18308, -149.92817 61.1...",2021
21,020200017022,"POLYGON ((-149.76341 61.19541, -149.76341 61.1...",2021
44,021300003003,"POLYGON ((-131.64374 55.33651, -131.64326 55.3...",2021


In [80]:
BG_list_df2012 = temp[temp['GEOID'].isin(common_bg)]

In [81]:
BG_list_df2012.head()

,GEOID,geometry,year
3,021850002002,"POLYGON ((-159.27452 69.26329, -159.27510 69.2...",2012
8,021800002002,"POLYGON ((-166.21494 64.43020, -166.27196 64.4...",2012
9,021800002003,"POLYGON ((-165.40501 64.49672, -165.40866 64.4...",2012
11,021800001001,"MULTIPOLYGON (((-168.97190 65.68877, -168.9717...",2012
17,021800002004,"POLYGON ((-165.38872 64.49445, -165.39153 64.4...",2012


In [130]:
BG_list_df2010 = temp[temp['GEOID'].isin(common_bg)]

In [131]:
BG_list_df2010.head()

,GEOID,geometry,year
9,022700001001,"POLYGON ((-162.54863 62.63832, -162.54863 62.6...",2010
14,021850002002,"POLYGON ((-156.91484 69.63572, -156.91575 69.6...",2010
28,021300002002,"POLYGON ((-131.68335 55.35728, -131.68358 55.3...",2010
36,021300003003,"POLYGON ((-131.63674 55.34341, -131.63663 55.3...",2010
59,021880002004,"POLYGON ((-162.50725 66.69428, -162.58721 66.7...",2010


In [145]:
BG_list_df2000 = temp[temp['GEOID'].isin(common_bg)]

In [146]:
BG_list_df2000.head()

,GEOID,geometry,year
18,022700001001,"POLYGON ((-162.54863 62.63832, -162.54863 62.6...",2000
23,021850002002,"POLYGON ((-156.91484 69.63572, -156.91575 69.6...",2000
36,021300003003,"POLYGON ((-131.63674 55.34341, -131.63663 55.3...",2000
37,021300002002,"POLYGON ((-131.68489 55.35622, -131.68469 55.3...",2000
64,021880002004,"POLYGON ((-162.50725 66.69428, -162.58721 66.7...",2000


In [160]:
BG_list_df1990 = temp[temp['GEOID'].isin(common_bg)]

In [161]:
BG_list_df1990.head()

,GEOID,geometry,year
86,020900019001,"POLYGON ((-145.94246 65.20531, -145.89345 65.2...",1990
89,020900017002,"POLYGON ((-144.05807 64.68399, -144.04901 64.6...",1990
91,020900019002,"POLYGON ((-146.96365 65.05630, -146.96227 65.0...",1990
96,020900019003,"POLYGON ((-147.54175 64.97759, -147.52670 64.9...",1990
98,020900012001,"POLYGON ((-147.63380 64.94161, -147.63335 64.9...",1990


In [162]:
BG_df = pd.concat([BG_list_df2021, BG_list_df2012, BG_list_df2010, BG_list_df2000, BG_list_df1990], ignore_index=True)

C:\Users\Asus\anaconda3\lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 (the single non-null crs provided).
  warnings.warn(


In [164]:
BG_df = BG_df.drop_duplicates(subset=['GEOID', 'geometry'])

In [165]:
BG_df.shape[0]

386324

In [168]:
BG_df[BG_df['GEOID'] == '780309604001']

,GEOID,geometry,year
78435,780309604001,"MULTIPOLYGON (((-64.90133 18.38485, -64.90108 ...",2021
183374,780309604001,"MULTIPOLYGON (((-64.90133 18.38485, -64.90108 ...",2012
289336,780309604001,"MULTIPOLYGON (((-64.90968 18.41509, -64.90960 ...",2010
363747,780309604001,"MULTIPOLYGON (((-64.90835 18.41266, -64.90824 ...",2000
421802,780309604001,"POLYGON ((-64.90532 18.40828, -64.90935 18.410...",1990
421803,780309604001,"POLYGON ((-64.90747 18.38966, -64.90890 18.390...",1990
421806,780309604001,"POLYGON ((-64.90944 18.36029, -64.91244 18.361...",1990


In [167]:
BG_df['GEOID'].value_counts()

780309604001    7
130390102003    7
530299718001    7
780309605004    7
530579501001    7
               ..
060470009032    1
060470016031    1
060650418033    1
060371438011    1
120850003005    1
Name: GEOID, Length: 106781, dtype: int64

In [169]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(BG_df), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = BG_df.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip", compression='gzip')

C:\Users\Asus\anaconda3\lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\Asus\anaconda3\lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\Asus\anaconda3\lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\Asus\anaconda3\lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\Asus\anaconda3\lib\site-packages\geopandas\geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\Asus\anaconda3\lib\site-package

In [ ]:
BG_df = pd.concat(BG_list, ignore_index=True)

In [ ]:
BG_df.head()

In [ ]:
list_censusBG_2 = BG_df['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG_2 = [censusBG.zfill(12) for censusBG in list_censusBG_2]

In [ ]:
len(list_censusBG_2)

In [ ]:
list_censusBG_3 = list_censusBG_3 + list_censusBG_2 

In [ ]:
len(list_censusBG_3)

In [ ]:
list_censusBG_3 = list(set(list_censusBG_3))

In [ ]:
len(list_censusBG_3)

In [ ]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_3)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

In [ ]:
len(common_bg)/len(list_censusBG)

In [ ]:
len(common_bg)

In [ ]:
len(list_censusBG)

## Using Shapefiles

In [ ]:
# Read the shapefile
shapefile_path = 'C:/Users/Asus/Box/Flood Damage PredictionProject/Shapefiles/tl_rd22_us_zcta520/tl_rd22_us_zcta520.shp'  # Replace with the path to your shapefile
map_df = gpd.read_file(shapefile_path)

In [ ]:
map_df.head()

In [ ]:
list_zipcode_2 = map_df['ZCTA5CE20'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_2 = [zipcode.zfill(5) for zipcode in list_zipcode]

In [ ]:
# Convert both lists to sets
set_zipcode = set(list_zipcode)
set_zipcode_2 = set(list_zipcode_2)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_2)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(common_zipcodes)/len(list_zipcode)